<a href="https://colab.research.google.com/github/Highpointer/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-18 01:38:42--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.97MB/s    in 0.2s    

2022-09-18 01:38:42 (4.97 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://highpointer.s3.amazonaws.com/amazon_reviews_us_Outdoors_v1_00.tsv" # replace this url
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Outdoors_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show(50, truncate=False)

+-----------+-----------+--------------+----------+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [6]:
# Create the vine_table DataFrame
#vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"]).withColumnRenamed("review_id", "Review ID").withColumnRenamed("star_rating", "Star Rating").withColumnRenamed("helpful_votes", "Helpful Votes").withColumnRenamed("total_votes", "Total Votes").withColumnRenamed("vine", "Vine").withColumnRenamed("verified_purchase", "Verified Purchase")
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(20, truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|4          |0            |0          |N   |Y                |
|R2BV735O46BN33|5          |0            |0          |N   |Y                |
|R2NBEUGPQQGXP1|4          |0            |0          |N   |Y                |
|R17LLAOJ8ITK0S|3          |1            |1          |N   |Y                |
|R39PEQBT5ISEF4|1          |0            |0          |N   |Y                |
|R3GNM3SU9VHJFT|4          |1            |1          |N   |Y                |
|R2Y81OP0EK467 |5          |0            |0          |N   |Y                |
|R2LFGSI6HAYH5F|5          |1            |1          |N   |Y                |
|R297G6ED1IQO7W|5          |1            |1          |N   |Y                |
|RE27RFC6101N6 |5          |0            |0          |N   |Y    

In [18]:
# Retrieve rows with 20 or more votes
vine_df_20reviews = vine_df.filter("total_votes>=20")
vine_df_20reviews.show(38, truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|5          |25           |29         |N   |Y                |
|R1UUK1977O38MU|5          |31           |31         |N   |Y                |
|RXO216LWUDV6O |3          |29           |31         |N   |Y                |
|R3NMJF7EBMM19V|3          |26           |27         |N   |Y                |
|R2ZY0ZBDUO0XUY|3          |20           |21         |N   |Y                |
|RBDVHX7PMQEKM |5          |35           |39         |N   |Y                |
|RGO3KSDXYWTGU |4          |26           |26         |N   |Y                |
|R2O4P39D0ALCQK|2          |14           |43         |N   |Y                |
|R1E1PQGHWGO3Q1|1          |89           |96         |N   |Y                |
|R1XOXXUVYZEDJB|5          |22           |22         |N   |Y    

In [14]:
# Retrieve rows where ratio of helpful votes to total votes is 50% or higher
vine_df_50percent_helpful = vine_df_20reviews.filter("helpful_votes/total_votes>=0.5")
vine_df_50percent_helpful.show(36, truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|5          |25           |29         |N   |Y                |
|R1UUK1977O38MU|5          |31           |31         |N   |Y                |
|RXO216LWUDV6O |3          |29           |31         |N   |Y                |
|R3NMJF7EBMM19V|3          |26           |27         |N   |Y                |
|R2ZY0ZBDUO0XUY|3          |20           |21         |N   |Y                |
|RBDVHX7PMQEKM |5          |35           |39         |N   |Y                |
|RGO3KSDXYWTGU |4          |26           |26         |N   |Y                |
|R1E1PQGHWGO3Q1|1          |89           |96         |N   |Y                |
|R1XOXXUVYZEDJB|5          |22           |22         |N   |Y                |
|R7SHITUU5UG9U |5          |27           |27         |N   |Y    

In [21]:
# Retrieve rows where ratio of helpful votes to total votes is 50% or higher
# and where the review was written as part of the Vine program (paid)
vine_df_50percent_vine = vine_df_50percent_helpful.filter("vine=='Y'")
vine_df_50percent_vine.show(36, truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KPC0NBUDASX3|5          |25           |25         |Y   |N                |
|R119P2A95GGXX4|5          |26           |28         |Y   |N                |
|R1HKIRME8AJ89Z|5          |79           |82         |Y   |N                |
|R3FY3GMBGOBR22|5          |12           |20         |Y   |N                |
|R15KH3FBSVYGBU|5          |32           |37         |Y   |N                |
|R22XZEQLVLWW49|5          |28           |33         |Y   |N                |
|R2OIQYVAL9D63R|4          |59           |65         |Y   |N                |
|R1W3KCDHM6BMZT|4          |37           |38         |Y   |N                |
|R3KDMPS96IJ10R|2          |15           |21         |Y   |N                |
|R1XWEI58A7VKLB|5          |123          |128        |Y   |N    

In [22]:
# Retrieve rows where ratio of helpful votes to total votes is 50% or higher
# and where the review was not written as part of the Vine program (unpaid)
vine_df_50percent_not_vine = vine_df_50percent_helpful.filter("vine<>'Y'")
vine_df_50percent_not_vine.show(36, truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|5          |25           |29         |N   |Y                |
|R1UUK1977O38MU|5          |31           |31         |N   |Y                |
|RXO216LWUDV6O |3          |29           |31         |N   |Y                |
|R3NMJF7EBMM19V|3          |26           |27         |N   |Y                |
|R2ZY0ZBDUO0XUY|3          |20           |21         |N   |Y                |
|RBDVHX7PMQEKM |5          |35           |39         |N   |Y                |
|RGO3KSDXYWTGU |4          |26           |26         |N   |Y                |
|R1E1PQGHWGO3Q1|1          |89           |96         |N   |Y                |
|R1XOXXUVYZEDJB|5          |22           |22         |N   |Y                |
|R7SHITUU5UG9U |5          |27           |27         |N   |Y    

In [23]:
vine_df_50percent_vine.show(10, truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KPC0NBUDASX3|5          |25           |25         |Y   |N                |
|R119P2A95GGXX4|5          |26           |28         |Y   |N                |
|R1HKIRME8AJ89Z|5          |79           |82         |Y   |N                |
|R3FY3GMBGOBR22|5          |12           |20         |Y   |N                |
|R15KH3FBSVYGBU|5          |32           |37         |Y   |N                |
|R22XZEQLVLWW49|5          |28           |33         |Y   |N                |
|R2OIQYVAL9D63R|4          |59           |65         |Y   |N                |
|R1W3KCDHM6BMZT|4          |37           |38         |Y   |N                |
|R3KDMPS96IJ10R|2          |15           |21         |Y   |N                |
|R1XWEI58A7VKLB|5          |123          |128        |Y   |N    

In [30]:
vine_df_count = vine_df_50percent_helpful.groupby("vine", "star_rating").count()
vine_df_count.show()

+----+-----------+-----+
|vine|star_rating|count|
+----+-----------+-----+
|   N|          1| 5324|
|   N|          5|21005|
|   Y|          5|   56|
|   N|          4| 7384|
|   N|          2| 2412|
|   Y|          4|   32|
|   N|          3| 3744|
|   Y|          2|    2|
|   Y|          3|   16|
|   Y|          1|    1|
+----+-----------+-----+



In [40]:
vine_df_count.orderBy("vine", "star_rating").show()

+----+-----------+-----+
|vine|star_rating|count|
+----+-----------+-----+
|   N|          1| 5324|
|   N|          2| 2412|
|   N|          3| 3744|
|   N|          4| 7384|
|   N|          5|21005|
|   Y|          1|    1|
|   Y|          2|    2|
|   Y|          3|   16|
|   Y|          4|   32|
|   Y|          5|   56|
+----+-----------+-----+



In [42]:
vine_count = vine_df_50percent_helpful.groupby("vine").count().orderBy("vine").show()

+----+-----+
|vine|count|
+----+-----+
|   N|39869|
|   Y|  107|
+----+-----+

